In [2]:
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath("../"))
from dataset import DS_DIMENSIONS, DS_NAMES
from utilities.figure_tools import (get_latest_results_directory, load_results,
                                    make_fig_directory)

cmap = plt.get_cmap("Set1")
colors = cmap.colors

results_path = "../results/compare_methods_turing.csv"
output_dir = "./compare-methods-turing/"
export_figures = False

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [3]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 10]
all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
print("explainers:", list(all_results["explainer"].unique()))
print("datasets", list(all_results["dataset"].unique()))
all_results.head(100)

explainers: ['AFT', 'FACETIndex', 'MACE', 'OCEAN', 'RFOCSE']
datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_manhattan,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,AFT,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,0.37,0.194152,0.256423,1.860480,0.000002,0.038712,0.001936,20.0
1,cancer,FACETIndex,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.153822,0.279627,4.330000,2.753081,1.884051,0.094203,20.0
2,cancer,MACE,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,1.365519,5.931730,30.000000,0.004978,44.437281,2.221864,20.0
3,cancer,OCEAN,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.156526,0.401209,11.410000,0.183316,13.617410,0.680870,20.0
4,cancer,RFOCSE,10.0,5.0,0.0,0.938596,0.950000,0.883721,0.915663,1.00,0.164461,0.360034,4.650000,0.027238,38720.307172,1936.015359,20.0
5,glass,AFT,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.00,0.085238,0.101321,1.485000,0.000002,0.035720,0.001786,20.0
6,glass,FACETIndex,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.00,0.085770,0.129234,2.755000,2.695310,0.113400,0.005670,20.0
7,glass,MACE,10.0,5.0,4.0,0.818182,0.829998,0.864772,0.842132,1.00,0.749309,1.671404,8.966667,0.003197,5050.227301,252.511365,20.0
8,glass,OCEAN,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.00,0.081336,0.157167,4.290000,0.199714,22.264149,1.113207,20.0
9,glass,RFOCSE,10.0,5.0,0.0,0.787879,0.800000,0.842105,0.820513,1.00,0.087032,0.101908,2.150000,0.012913,75.539079,3.776954,20.0


In [4]:
# facet_idx = (all_results["explainer"] == "FACETIndex")
# all_results[facet_idx]["avg_dist"].head()
# all_results["norm_dist"] = all_results["avg_dist"] / all_results[facet_idx]["avg_dist"]
all_results.head()

,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_manhattan,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,AFT,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,0.37,0.194152,0.256423,1.86048,0.000002,0.038712,0.001936,20.0
1,cancer,FACETIndex,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.153822,0.279627,4.33000,2.753081,1.884051,0.094203,20.0
2,cancer,MACE,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,1.365519,5.931730,30.00000,0.004978,44.437281,2.221864,20.0
3,cancer,OCEAN,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.156526,0.401209,11.41000,0.183316,13.617410,0.680870,20.0
4,cancer,RFOCSE,10.0,5.0,0.0,0.938596,0.950000,0.883721,0.915663,1.00,0.164461,0.360034,4.65000,0.027238,38720.307172,1936.015359,20.0


In [5]:
# df = all_results.copy() #.set_index(["explainer", "dataset"])
# df.reset_index()
# for expl in all_results["explainer"].unique():
#     for ds in all_results["dataset"].unique():
#         facet_idx = (df["explainer"] == "FACETIndex") & (df["dataset"] == ds)
#         expl_idx = (df["explainer"] == expl) & (df["dataset"] == ds)    
#         df.loc[expl_idx, "avg_dist"] = float(df.loc[expl_idx, "avg_dist"]) / float(df.loc[facet_idx, "avg_dist"])
#         print()
#         print(df.iloc[facet_idx])
#         print(df.loc[[expl, ds]]["avg_dist"] / df.loc[["FACETIndex", ds]]["avg_dist"])
#         print(df.loc[expl, ds]["norm_dist"])
#         df.loc[expl, "avg_dist"] = df.loc[expl, "avg_dist"].div(df.loc["FACETIndex", "avg_dist"])
#     print()
# df.head(100)
# df[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

In [6]:
# facet_dists.head()
norm_data = all_results.copy()
for expl in norm_data["explainer"].unique():
    for ds in norm_data["dataset"].unique():
        facet_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
#         facet_dist = all_results[facet_idx]["avg_dist"]

#         match_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
#         norm_data[match_idx] = norm_data[match_idx] / facet_dist

# norm_data["avg_dist"] = norm_data["avg_dist"] / facet_dists
# norm_data.head()

In [130]:
opt_vals = {}
bold_opt = False
domin = {
    "sample_time": True,
    "avg_length": True,
    "avg_dist": True,
    "avg_manhattan": True,
    "per_valid": False,
}
metrics = ["sample_time", "avg_length", "avg_dist", "avg_manhattan", "per_valid"]


for ds in all_results["dataset"].unique():
    idx = (all_results["dataset"] == ds)
    opt_vals[ds] = {}
    for m in metrics:
        if domin[m]:
            opt_vals[ds][m] = all_results[idx][m].min()
        else:
            opt_vals[ds][m] = all_results[idx][m].max()

metric_latex = {
    "sample_time": "$t\downarrow$",
    "avg_length": "$\delta_0\downarrow$",
    "avg_manhattan": "$\delta_1\downarrow$",
    "avg_dist": "$\delta_2\downarrow$",
    "per_valid": "$\%\\uparrow$",
}
pretty_names = {
    "FACETIndex": "FACET",
    "MACE": "MACE",
    "AFT": "AFT",
    "OCEAN": "OCEAN",
    "RFOCSE": "RFOCSE",
}
all_metrics = ["sample_time", "avg_length", "avg_manhattan", "avg_dist", "per_valid"]

In [147]:
def df_to_csv_latex(df_source, expls, metrics, fname, include_row_label=True):
    df_ds = df_source["dataset"].unique()
    df = df_source.copy()
    df = df.set_index(["dataset", "explainer"])
    with open(fname + ".csv", "w") as csv:
        with open(fname + ".tex", "w") as tab:
            # csv header row start
            if include_row_label:
                csv.write("dataset,")
            # tab header row start
            tab.write("\\begin{table*}[t]\n\small\n\centering\n\\begin{tabularx}{0.95\\textwidth}{")
            if include_row_label:
                tab.write("|X")
            tab.write("|")
            for expl in expls:
                for m in metrics:
                    tab.write("c")
                tab.write("|")
            tab.write("}\n\hline")
            if include_row_label:
                tab.write("\\textbf{Dataset}")
            # csv and tab header row
            for expl in expls:
                tab.write(" & \multicolumn{5}{c|}{\\textbf{" + pretty_names[expl] +"}}")
                for m in metrics:
                    csv.write(pretty_names[expl] + ",")
                csv.write(",")
            csv.write("\n")
            tab.write(" " + re.escape("\\") + "\n")
            # header row two
            csv.write(",")
            for expl in expls:
                for m in metrics:
                    csv.write(m + ",")
                    tab.write("& " + metric_latex[m] + " ")
                csv.write(",")
            csv.write("\n")
            tab.write(re.escape("\\") + "\n\hline\n")
            # csv and tab body row
            for ds in df_ds:
                if include_row_label:
                    csv.write(ds + ",")
                    tab.write(ds)
                for expl in expls:
                    for m in metrics:
                        if m == "per_valid":
                            val = df.loc[ds, expl][m] * 100
                            val_str = "{:0.1f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_length":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_dist":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "avg_manhattan":
                            val = df.loc[ds, expl][m] / df.loc[ds, "FACETIndex"][m]
                            val_str = "{:0.2f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        elif m == "sample_time":
                            val = df.loc[ds, expl][m]
                            val_str = "{:0.4f}".format(val)
                            csv.write(val_str + ",")
                            if bold_opt and df.loc[ds, expl][m] == opt_vals[ds][m]:
                                tab.write(" & \\textbf{" + val_str + "}")
                            else:
                                tab.write(" & " + val_str)
                        else:
                            val_str = str(df.loc[ds, expl][m])
                            csv.write(val_str + ",")
                            tab.write(" & " + val_str)
                    csv.write(",")
                csv.write("\n")
                tab.write(" " + re.escape("\\") + "\n")
            # tab latex close
            tab.write("\hline\n\end{tabularx}\n")
            tab.write("\caption{Comparison to state-of-the art counterfactual example generation techniques in terms of explanation time $t$, explanation distance $\delta$, and percent of instances successfully explained. ($\\ast$) denotes cases which necessitated uncapped explanation time.}\n")
            tab.write("\label{tab.compare_methods}\n\\vspace{-7mm}\n\end{table*}\n")

In [148]:
idx1 = all_results["explainer"].isin(["FACETIndex", "MACE", "OCEAN"])
idx2 = all_results["explainer"].isin(["RFOCSE", "AFT"])
df_to_csv_latex(all_results, ["FACETIndex", "MACE", "OCEAN"], all_metrics, "compare_methods_1")
df_to_csv_latex(all_results, ["RFOCSE", "AFT"], all_metrics, "compare_methods_2", False)

In [8]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["per_valid"])

per_valid                             
explainer       AFT FACETIndex MACE OCEAN RFOCSE
dataset                                         
cancer         0.37        1.0  1.0   1.0   1.00
glass          1.00        1.0  1.0   1.0   1.00
magic          0.99        1.0  1.0   1.0   0.95
spambase       0.76        1.0  1.0   1.0   1.00
vertebral      1.00        1.0  1.0   1.0   1.00

In [9]:
all_results[""]

KeyError: ''

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

avg_dist                                         
explainer       AFT FACETIndex      MACE     OCEAN    RFOCSE
dataset                                                     
cancer     0.194152   0.153822  1.365519  0.156526  0.164461
glass      0.085238   0.085770  0.753847  0.081336  0.055554
magic      0.118433   0.095715  0.728178  0.100905  0.075638
spambase   0.024882   0.008207  3.042850  0.005249  0.037012
vertebral  0.080469   0.066099  0.454783  0.072163  0.059878

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["sample_time"])

sample_time                                             
explainer         AFT FACETIndex       MACE     OCEAN       RFOCSE
dataset                                                           
cancer       0.001804   0.020463   2.114727  0.914013  2001.016770
glass        0.001745   0.001050  10.754155  1.387624     2.833081
magic        0.003289   0.001736   2.267405  2.246202    70.124059
spambase     0.002600   0.015470  20.438034  0.585131  2249.782774
vertebral    0.001922   0.001436   3.858280  1.214346     4.099172

In [ ]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 100]
# all_results["diff"] = all_results["sample_time"] - (((all_results["sample_time"] * all_results["n_explain"]) - ((1 - all_results["per_valid"]) * all_results["n_explain"] * 300)) / all_results["n_explain"])
tot_time = all_results["sample_time"] * all_results["n_explain"]
n_timeout = all_results["per_valid"] * all_results["n_explain"]
max_time = 300
timeout_cost = n_timeout * max_time
all_results["adj_time"] = (tot_time - timeout_cost) / all_results["n_explain"]
# all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
# print("explainers:", list(all_results["explainer"].unique()))
# print("datasets", list(all_results["dataset"].unique()))
all_results.head()

,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain,adj_time
